# import packages

In [228]:
#libraries and dirrectory 
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy.optimize import curve_fit
import patientFunctions as ptfn
import seaborn as sns
import matplotlib.pyplot as plt
import re

# directory management

In [229]:
#location of original data
starting_directory = os.getcwd()


machine_directory = 'C:/Users/mcremer' #the C and path to the project folder
# machine_directory = 'C:/Users/maega' #when working from home machine
storage_directory = 'UFL Dropbox/Maegan Cremer/research-share/Maegan/Projects' #Local, HPG, or dropbox
project_directory = 'Cardiac-Amyloidosis-Multiple-Myeloma' #project folder
project_lv2_directory = '012 Processed Data/data-2024-06-05' #deeper part of project folder

path = os.path.join(machine_directory, storage_directory, 
                    project_directory, project_lv2_directory)

new_dirrectory = path

os.chdir(new_dirrectory)

In [230]:
#updates to directory management
machine_directory = 'C:/Users/mcremer' #the C and path to the project folder
# machine_directory = 'C:/Users/maega' #when working from home machine
storage_directory = 'UFL Dropbox/Maegan Cremer/research-share/Maegan/Projects' #Local, HPG, or dropbox
project_directory = 'Cardiac-Amyloidosis-Multiple-Myeloma' #project folder
project_lv2_directory = '015_Mistic2025' #deeper part of project folder

path = os.path.join(machine_directory, storage_directory, 
                    project_directory, project_lv2_directory)

# parent_dir = path
outputDir = path

## naming file outputs

In [231]:
folderNameHeader = "DFsForSVM"
folderNameCommon = ""
folderNameSuffix = "_20250402_v1"

fileNameHeader_1 = "DescFits"
# fileNameHeader_2 = "ALUnknownPts"
fileNameCommon = "_Features"
fileNameSuffix = "_20250402_v1"

folder_data_out = ptfn.makeFolderPathForData(parent_dir=outputDir, folderName_header=folderNameHeader, 
                                        folderName_common=folderNameCommon, folderName_suffix=folderNameSuffix)
# outputToExcel(*, fileName_header=fileNameHeader, fileName_Common= fileNameCommon, 
#               fileName_suffix= fileNameSuffix, folderName = folder_data_out, sheetName= *)

# fig_file_out = outputFiguresPath(fileName_header=fileNameHeader, fileName_mid=**, 
#                                  fileName_suff= fileNameSuffix, parent_dir=parent_dir, folderName=folder_data_out)

folderNameOut = folder_data_out

In [232]:
#file name suffix 
outputSheet_Suffix = fileNameSuffix
outputSheet_common = fileNameCommon

# Penalty scores used throughout for missing data and low time resolution

## length 1

In [233]:
#filler information for fits of 1 point
# one_point_STD = 0
one_point_r2 = -1
# one_point_residual_error = 0

## length 0

In [234]:
penalty_coef= np.nan
penalty_intercept = np.nan
penalty_R2 = np.nan
penalty_residual_error = np.nan
penalty_fillna = np.nan

In [235]:
penalty_echo = -1


# collecting patient information

In [237]:
#importing patient data as dictionaries of dataframes

#build a dictionary of patients and their values 
ptFiles = ptfn.GetListofPTfiles(new_dirrectory, '.xlsx', 'data')

ptDict = {}

#loop through the listOfPatients 
for file in ptFiles: 
    pt = ptfn.patient(new_dirrectory, file, 4)
    ptDict[pt.ptID] = pt

In [238]:
EchoTimeColumn ='time from diagnosis to first echo (months)'
MRITimeColumn = 'time from dagnosis to MRI (months)'
echoDates = {}
for pt in ptDict.keys():
    ptEchoData = ptDict[pt].echoData
    patientEchoDate = ptEchoData[EchoTimeColumn]
    if 'none' not in list(patientEchoDate):
        echoDates[pt] = patientEchoDate

#test to see if this still runs and if the data is correct

In [ ]:
# EchoTimeColumn ='time from diagnosis to first echo (months)'
# MRITimeColumn = 'time from dagnosis to MRI (months)'
# #convert the echoDates from months to days
# imagingDates_days = {}
# for pt in ptDict.keys():
#     ptEchoData = ptDict[pt].echoData
#     patientEchoDate = ptEchoData[EchoTimeColumn]
#     #if the patientEchoDate is not 'none' then convert to days
#     if 'none' not in list(patientEchoDate):
#         ptImageDate = patientEchoDate * 30.44
#         imagingDates_days[pt] = ptImageDate

#     #if the patientEchoDate is 'none' then find time "time from dagnosis to MRI (months)" and convert to days
#     if 'none' in list(patientEchoDate):
#         patientMRIDate = ptEchoData[MRITimeColumn]
#         if 'none' not in list(patientMRIDate):
#             ptImageDate = patientMRIDate * 30.44
#             imagingDates_days[pt] = ptImageDate

# collecting patients of interest

In [239]:
#list of patients by amyloid status 

listPositive = []
listNegative = []
listOther = []


list_pts_remove = ["K-01", "G-01", "J-01", "L-02"]
#remove list_pts_remove from ptDict
for pt in list_pts_remove:
    if pt in ptDict:
        del ptDict[pt]

#get list of positive patients 
for pt in ptDict:
    ptALStatus = ptDict[pt].amyloid_status
    if ptALStatus == 'yes':
        listPositive.append(pt)
    elif ptALStatus == 'no':
        listNegative.append(pt)
    else:
        listOther.append(pt)


In [240]:

listKnown = listPositive + listNegative

# collecting the labs of interest

labs of interest have at least one data point for each patient and aren't tagged as methods or limits of normality

In [241]:
# First cleaning the lab names while keeping 'RelTime(Days)' unchanged
cleanLabNames = [re.sub(r'\(.*?\)', '', x).replace("Result", '').replace('.', '').strip() if x != 'RelTime(Days)' else x for x in ptDict['E-01'].labsData.columns]

In [242]:
#apply the clean lab names to all patients labsData
for pt in ptDict:
    ptDict[pt].labsData.columns = cleanLabNames

In [243]:
#for patient labs, convert to boolean and find the number of each lab (true)
ptLabsBool = {}
for pt in ptDict.keys():
    ptLabsData = ptDict[pt].labsData
    #convert to boolean if not nan
    ptLabsData = ptLabsData.notna() #not sure this is capturing the null data correctly
    #find the number of each lab (true)
    ptLabsData = ptLabsData.sum(axis=0) #check the axis
    #add to the dictionary describing the number of each lab the patient has
    ptLabsBool[pt] = ptLabsData

#then find frequency of labs across all patients
ptLabsBool_df = pd.DataFrame(ptLabsBool)
#count frequency of value in a column 

dict_value_counts_labs ={}
for lab in ptLabsBool_df.index:
    #call a row and perform value counts, save it to a key as the lab name in the dictionary
    dict_value_counts_labs[lab] = ptLabsBool_df.loc[lab].value_counts()

#convert to dataframe and transpose
df_value_counts_labs = pd.DataFrame(dict_value_counts_labs).T

In [244]:
#removing things that aren't lab values
omit = ['Method',"Lower Limit of Normal", "Upper Limit of Normal"]

#drop the rows whos string contains items from the omit list
df_value_counts_labs_filtered = df_value_counts_labs[~df_value_counts_labs.index.str.contains('|'.join(omit))]

#collect the rows where the column"0" is null
df_value_counts_labs_filtered = df_value_counts_labs_filtered[df_value_counts_labs_filtered[0].isnull()]

#print the index of df_value_counts_labs_filtered
filteredLabs = df_value_counts_labs_filtered.index.tolist()

In [245]:
#remove 'DeID', 'Survival(Mo)', 'RelTime(Days)', 'Unnamed: 0' from the filteredLabs list
filteredLabs = [x for x in filteredLabs if x not in ['DeID', 'Survival', 'RelTime(Days)', 'Unnamed: 0']]

In [246]:
filteredLabs

['ALT',
 'AST',
 'Albumin Electrophoresis',
 'Albumin',
 'BUN',
 'Beta-2-Microglobulin',
 'Calcium',
 'Chloride',
 'Creatinine',
 'Glucose',
 'HCT',
 'Hemoglobin',
 'Immature Gran Auto',
 'Kappa Free Light Chains',
 'Kappa/Lambda Free Light Chain Ratio',
 'LDH',
 'Lambda Free Light Chains',
 'MCH',
 'MCHC',
 'MCV',
 'Monocyte',
 'Neutrophil',
 'Nucleated RBC',
 'Pct Immature Gran Auto',
 'Pct Mono',
 'Pct Neutrophil',
 'Platelet Count',
 'Potassium',
 'RBC',
 'Serum Electrophoresis Alpha 1',
 'Serum Electrophoresis Alpha 2',
 'Serum Electrophoresis Beta',
 'Serum Electrophoresis Gamma',
 'Sodium',
 'Total Serum Protein',
 'WBC',
 'total CO2',
 'NT Pro BNP',
 'Troponin-T']

In [247]:
list(ptDict['E-01'].labsData.columns)

['Unnamed: 0',
 'DeID',
 '5-HIAA',
 'Anisocytosis',
 'Macrocytes',
 '25 Hydroxy Vitamin D',
 'A1C',
 'ABG Base Excess',
 'ABG pH',
 'ABORh Type',
 'ALT',
 'ALT Upper Limit of Normal',
 'APTT Lower Normal Limit',
 'APTT',
 'APTT Upper Normal Limit',
 'AST',
 'AST Upper Limit of Normal',
 'Albumin Electrophoresis',
 'Albumin Lower Limit of Normal',
 'Albumin',
 'Albumin Upper Limit of Normal',
 'Amylase',
 'Amylase Upper Limit of Normal',
 'Arterial Lactic Acid',
 'Atypical Lymphocyte',
 'Atypical Lymphocyte  Method',
 'B12 Lower Normal Limit',
 'B12',
 'B12 Upper Normal Limit',
 'RelTime(Days)',
 'Survival',
 'BUN',
 'BUN Upper Limit of Normal',
 'Basophil',
 'Basophil  Method',
 'Beta-2-Microglobulin Lower Limit of Normal',
 'Beta-2-Microglobulin',
 'Beta-2-Microglobulin Upper Limit of Normal',
 'Bilirubin Lower Limit of Normal',
 'Bilirubin',
 'Bilirubin Upper Limit of Normal',
 'Blast',
 'Blast  Method',
 'CPK Lower Normal Limit',
 'CPK',
 'CPK Upper Normal Limit',
 'Calcium Cation E

In [248]:
len(filteredLabs)

39

## slice of patient data near echo or MRI date

In [249]:
EchoTimeColumn ='time from diagnosis to first echo (months)'
MRITimeColumn = 'time from dagnosis to MRI (months)'
#convert the echoDates from months to days
imagingDates_days = {}
for pt in ptDict.keys():
    ptEchoData = ptDict[pt].echoData
    patientEchoDate = ptEchoData[EchoTimeColumn]
    #if the patientEchoDate is not 'none' then convert to days
    if 'none' not in list(patientEchoDate):
        ptImageDate = patientEchoDate * 30.44
        imagingDates_days[pt] = ptImageDate

    #if the patientEchoDate is 'none' then find time "time from dagnosis to MRI (months)" and convert to days
    if 'none' in list(patientEchoDate):
        patientMRIDate = ptEchoData[MRITimeColumn]
        if 'none' not in list(patientMRIDate):
            ptImageDate = patientMRIDate * 30.44
            imagingDates_days[pt] = ptImageDate

In [250]:
ptData_ImagingAdj = {}
for pt in listKnown: #uses only the list of known patients
    ptLabsData = ptDict[pt].labsData
    #call the row of data where the RelTime(Days) is closest to the value of imagingDates_days[pt]
    ptLabsData_slice = ptLabsData.loc[(ptLabsData['RelTime(Days)'] - imagingDates_days[pt]).abs().argsort()[:1]]
    ptData_ImagingAdj[pt] = ptLabsData_slice[filteredLabs].iloc[0]  #single row

#convert to dataframe
ptData_ImagingAdj_df = pd.DataFrame.from_dict(ptData_ImagingAdj, orient='index')

#export to excel as the EchoAdj labs

C:\Users\mcremer\AppData\Local\Temp\ipykernel_23108\602117363.py:5: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  ptLabsData_slice = ptLabsData.loc[(ptLabsData['RelTime(Days)'] - imagingDates_days[pt]).abs().argsort()[:1]]
C:\Users\mcremer\AppData\Local\Temp\ipykernel_23108\602117363.py:5: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  ptLabsData_slice = ptLabsData.loc[(ptLabsData['RelTime(Days)'] - imagingDates_days[pt]).abs().argsort()[:1]]
C:\Users\mcremer\AppData\Local\Temp\ipykernel_23108\602117363.py:5: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  ptLabsData_slice = ptLabsData.loc[(ptLabsData['RelTime(Days)'] - imagingDates_da

In [251]:
ptData_ImagingAdj_df.loc[listPositive]

,ALT,AST,Albumin Electrophoresis,Albumin,BUN,Beta-2-Microglobulin,Calcium,Chloride,Creatinine,Glucose,...,Serum Electrophoresis Alpha 1,Serum Electrophoresis Alpha 2,Serum Electrophoresis Beta,Serum Electrophoresis Gamma,Sodium,Total Serum Protein,WBC,total CO2,NT Pro BNP,Troponin-T
B-02,19.0,14.0,3.9,4.1,39.0,18.1,9.0,101.0,8.3,101.0,...,0.4,0.7,0.5,0.4,142.0,5.9,16.24,24.0,611.0,0.015
C-03,69.0,102.0,3.0,2.8,35.0,12.0,8.9,105.0,2.5,84.0,...,0.5,1.4,1.0,0.5,141.0,6.4,10.00,22.0,3321.0,0.492
E-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.79,NaN,NaN,NaN
E-02,28.0,22.0,2.9,2.4,34.0,7.5,8.4,99.0,3.9,167.0,...,0.3,1.5,0.6,0.2,136.0,5.5,8.69,21.0,9328.0,0.088
F-03,7.0,12.0,4.3,4.1,29.0,3.8,9.5,101.0,1.3,91.0,...,0.3,0.9,0.9,1.3,139.0,7.7,3.54,27.0,166.0,<0.010
G-02,13.0,14.0,2.7,3.2,11.0,4.0,9.5,104.0,1.1,95.0,...,0.6,1.2,0.6,0.3,142.0,5.3,6.48,22.0,5928.0,0.086
I-01,19.0,29.0,2.2,1.9,10.0,2.5,8.5,101.0,0.8,104.0,...,0.3,1.5,0.7,0.3,139.0,5.0,7.34,28.0,309.0,<0.010
I-03,24.0,30.0,51.4,1.8,16.0,2.6,8.2,107.0,1.2,84.0,...,4.8,22.4,14.0,7.4,141.0,3.8,5.12,27.0,314.0,<0.010
J-02,60.0,38.0,4.0,4.0,9.0,2.5,9.3,96.0,0.8,95.0,...,0.5,0.8,0.7,1.7,133.0,7.7,7.02,22.0,9293.0,0.019
K-02,10.0,22.0,2.9,3.0,47.0,8.0,9.3,104.0,3.5,94.0,...,0.3,0.8,1.5,0.8,142.0,6.3,9.44,24.0,10432.0,0.102


## clean the lab names for working with throughout the document

In [252]:
#further clean the labs list to remove units and other non-lab values

# encoding demographic information

In [253]:
import re

In [254]:
#collect age, sex, and dx and make it lowercase
#make a dictionary of patient demographics
ptSex, ptDx, ptALStatus = {}, {}, {}

for pt in ptDict.keys():
    ptSex[pt] = ptDict[pt].sex.lower()
    # ptAge[pt] = ptDict[pt].ageDx
    ptDx[pt] = ptDict[pt].dx
    ptDx[pt] = re.sub(r"[\\/\d]", "", ptDx[pt]).lower()
    ptALStatus[pt] = ptDict[pt].amyloid_status.lower()

    
    #Convert "multiple myeloma" to "MM"
    ptDx[pt] = ptDx[pt].replace("multiple myeloma", "mm")
    #still have compound entries
    # if ptDx[pt] contains "mg" replace the entry with "mgus", if contains "mm" replace with "mm"
    if "mg" in ptDx[pt]:
        ptDx[pt] = "mgus"
    elif "mm" in ptDx[pt]:
        ptDx[pt] = "mm"
    elif "mm" or "mg" not in ptDx[pt]:
        ptDx[pt] = 'other'

#convert the dictionaries to dataframes and concatenate them
ptSex_df = pd.DataFrame.from_dict(ptSex, orient='index', columns=['sex'])
ptALStatus_df = pd.DataFrame.from_dict(ptALStatus, orient='index', columns=['CA_status'])
# ptAge_df = pd.DataFrame.from_dict(ptAge, orient='index', columns=['age'])
ptDx_df = pd.DataFrame.from_dict(ptDx, orient='index', columns=['dx'])
#concatenate the dataframes
ptDemographics_df = pd.concat([ptALStatus_df, ptSex_df, ptDx_df], axis=1)

In [255]:
#onehot encoding of the categorical variables
ptDemographics_df_encoded = pd.get_dummies(ptDemographics_df, columns=["CA_status", "sex", "dx",], dtype=int, drop_first=True)

In [256]:
#collecting age and survival data
ptAge, ptsurvival = {}, {}
for pt in ptDict.keys():
    ptAge[pt] = ptDict[pt].ageDx
    ptsurvival[pt] = ptDict[pt].survival

#convert to dataframe
ptAge_df = pd.DataFrame.from_dict(ptAge, orient ='index', columns = ['age'])
ptsurvival_df = pd.DataFrame.from_dict(ptsurvival, orient = 'index', columns = ['survival(mo)'])
#concatenate the dataframes
ptDemographics_df = pd.concat([ptDemographics_df, ptAge_df, ptsurvival_df],
                               axis=1)
#rename the columns


# encoding note information

In [257]:
#dictionaries for conversion
dictECOGtoKPS = {0 : 95,
                 1 : 75, 
                 2 : 55, 
                 3 : 35, 
                 4 : 15, 
                 5 : 0}

dictKPStoECOG = {100: 0, 90: 0, 
                 80: 1, 70: 1, 
                 60: 2, 50: 2, 
                 40: 3 , 30: 3, 
                 20: 4, 10: 4, 0: 5 }

ptDictMDnotesScored = {}
#loop through the listOfPatients
for pt in ptDict.keys():
    #call each patient in the dictionary storing patient note information
    ptNotes = ptDict[pt].MDnotes

    if not isinstance(ptNotes, bool):

        #columns of interest
        listColumnNames = ['time from diagnosis to note (months)', 'ECOG Performance Status', 'Karnofsky Performance Status']
        ptNotesCorrected = ptNotes[listColumnNames]
        #convert the Karnofsky scores to integers
        ptNotesCorrected['Karnofsky Performance Status'] = ptNotesCorrected['Karnofsky Performance Status'].astype(str).str.extract('(\d+)').astype(float)

        #remove rows where the "ECOG Performance Status" is "OTHER"
        ptNotesCorrected = ptNotesCorrected[ptNotesCorrected['ECOG Performance Status'] != 'OTHER'] 
        #remove rows where the "Karnofsky Performance Status" is "OTHER"
        ptNotesCorrected = ptNotesCorrected[ptNotesCorrected['Karnofsky Performance Status'] != 'OTHER']

        # old methods, we will see if we still need this

        for i in list(ptNotesCorrected.index):
            #get the scores for that row
            ptTime = ptNotesCorrected.at[i, 'time from diagnosis to note (months)']
            ptECOG = ptNotesCorrected.at[i,'ECOG Performance Status']
            ptKPS = ptNotesCorrected.at[i, 'Karnofsky Performance Status']
        
            #if the value of ptECOG was "OTHER", then the row should be dropped.
            if ptECOG == "OTHER":
                # ptNotesCorrected.drop(index = i, inplace=True ) #drops the full row, would need to add time back if interested in clinical impression
                ptECOG = np.nan #replacement is dropped before descriptive stats are run
                ptKPS = np.nan #replacement is dropped before descriptive stats are run

            #infer ECOG from Karnofsky where you now have half points
            elif np.isnan(ptECOG) ==True and pd.isna(ptKPS) == False: 
                ptECOG = dictKPStoECOG[ptKPS]

            #infer Karnofsky from ECOG 
            elif pd.isna(ptKPS) == True and np.isnan(ptECOG) ==False: 
                ptKPS = dictECOGtoKPS[ptECOG]

            #save these values back to the dataframe
            if ptECOG != "OTHER":
                ptNotesCorrected.at[i,'ECOG Performance Status'] = ptECOG
                ptNotesCorrected.at[i, 'Karnofsky Performance Status'] = ptKPS
                                
    #saving the notes to the new dictionary
    ptDictMDnotesScored[pt] = ptNotesCorrected


<>:27: SyntaxWarning: invalid escape sequence '\d'
<>:27: SyntaxWarning: invalid escape sequence '\d'
C:\Users\mcremer\AppData\Local\Temp\ipykernel_23108\2233743332.py:27: SyntaxWarning: invalid escape sequence '\d'
  ptNotesCorrected['Karnofsky Performance Status'] = ptNotesCorrected['Karnofsky Performance Status'].astype(str).str.extract('(\d+)').astype(float)
C:\Users\mcremer\AppData\Local\Temp\ipykernel_23108\2233743332.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptNotesCorrected['Karnofsky Performance Status'] = ptNotesCorrected['Karnofsky Performance Status'].astype(str).str.extract('(\d+)').astype(float)
C:\Users\mcremer\AppData\Local\Temp\ipykernel_23108\2233743332.py:27: SettingWithCopyWarning: 
A value is trying to be set on a 

# descriptive stats for each lab by patient
don't forget to handle instances of 1 for values which aren't counts to be the one recorded value

In [259]:
# cleaning troponin where it is a string entry of '<0.010' or BNP has '<5' as an entry to 0
for pt in listKnown:
    ptDict[pt].labsData['Troponin-T'].replace('<0.010', 0, inplace=True)
    ptDict[pt].labsData['NT Pro BNP'].replace('<5', 0, inplace=True)


C:\Users\mcremer\AppData\Local\Temp\ipykernel_23108\3595993921.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ptDict[pt].labsData['Troponin-T'].replace('<0.010', 0, inplace=True)
C:\Users\mcremer\AppData\Local\Temp\ipykernel_23108\3595993921.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

In [260]:
#descriptive statistics for each lab for each patient and assign to a dictionary
ptLabsDescriptiveStats = {}
#loop through the list of known patients
for pt in listKnown:
    #call each patient in the dictionary storing patient lab information
    ptLabs = ptDict[pt].labsData[filteredLabs] #instead call the filtered labs situation
    #get the descriptive statistics for the labs
    ptLabsDescriptiveStats_calc = ptLabs[filteredLabs].describe()
    ptLabsDescriptiveStats_calc['stats'] = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
    #when the count of data is 1, then use the value of the lab as the std
    for lab in filteredLabs:
        if ptLabsDescriptiveStats_calc[lab]['count'] == 1:
            ptLabsDescriptiveStats_calc[lab]['std'] = ptLabsDescriptiveStats_calc[lab]['mean']
            ptLabsDescriptiveStats_calc[lab]['min'] = ptLabsDescriptiveStats_calc[lab]['mean']
            ptLabsDescriptiveStats_calc[lab]['25%'] = ptLabsDescriptiveStats_calc[lab]['mean']
            ptLabsDescriptiveStats_calc[lab]['50%'] = ptLabsDescriptiveStats_calc[lab]['mean']
            ptLabsDescriptiveStats_calc[lab]['75%'] = ptLabsDescriptiveStats_calc[lab]['mean']
            ptLabsDescriptiveStats_calc[lab]['max'] = ptLabsDescriptiveStats_calc[lab]['mean']

    #melt dataframe to long format 
    ptLabsDescriptiveStats_melt = ptLabsDescriptiveStats_calc[filteredLabs].T.stack(future_stack=True) 

    ptLabsDescriptiveStats[pt] = ptLabsDescriptiveStats_melt 

#convert to dataframe
ptLabsDescriptiveStats_df = pd.DataFrame(ptLabsDescriptiveStats).T #each row is a patient
# flattens the multiindex columns into a single index with underscore as delimiter
ptLabsDescriptiveStats_df.columns = ['_'.join(col).strip() for col in ptLabsDescriptiveStats_df.columns.values] 

C:\Users\mcremer\AppData\Local\Temp\ipykernel_23108\3247168963.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  ptLabsDescriptiveStats_calc[lab]['std'] = ptLabsDescriptiveStats_calc[lab]['mean']
C:\Users\mcremer\AppData\Local\Temp\ipyker

## descriptive stats of notes information

In [261]:
ptNotesDescriptiveStats = {}
noteColumns = ["ECOG Performance Status", "Karnofsky Performance Status"]
for pt in listKnown:
    ptNotes = ptDictMDnotesScored[pt][noteColumns].astype(float)
    #get the descriptive statistics for the notes
    ptNotesDescriptiveStats_calc = ptNotes.describe()
    ptNotesDescriptiveStats_calc['stats'] = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
    #when the count of data is 1, then use the value of the lab as the std
    for lab in noteColumns:
        if ptNotesDescriptiveStats_calc[lab]['count'] == 1:
            ptNotesDescriptiveStats_calc[lab]['std'] = ptNotesDescriptiveStats_calc[lab]['mean']
            ptNotesDescriptiveStats_calc[lab]['min'] = ptNotesDescriptiveStats_calc[lab]['mean']
            ptNotesDescriptiveStats_calc[lab]['25%'] = ptNotesDescriptiveStats_calc[lab]['mean']
            ptNotesDescriptiveStats_calc[lab]['50%'] = ptNotesDescriptiveStats_calc[lab]['mean']
            ptNotesDescriptiveStats_calc[lab]['75%'] = ptNotesDescriptiveStats_calc[lab]['mean']
            ptNotesDescriptiveStats_calc[lab]['max'] = ptNotesDescriptiveStats_calc[lab]['mean']
    
    #melt to long format
    ptNotesDescriptiveStats_melt = ptNotesDescriptiveStats_calc[noteColumns].T.stack(future_stack=True)
    ptNotesDescriptiveStats[pt] = ptNotesDescriptiveStats_melt

#make to dataframe
ptNotesDescriptiveStats_df = pd.DataFrame(ptNotesDescriptiveStats).T #each row is a patient
# flattens the multiindex columns into a single index with underscore as delimiter
ptNotesDescriptiveStats_df.columns = ['_'.join(col).strip() for col in ptNotesDescriptiveStats_df.columns.values]

C:\Users\mcremer\AppData\Local\Temp\ipykernel_23108\2838012962.py:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  ptNotesDescriptiveStats_calc[lab]['std'] = ptNotesDescriptiveStats_calc[lab]['mean']
C:\Users\mcremer\AppData\Local\Temp\ipyk

# treatment information

In [262]:
#collect the number of treatments of different types a patient has recieved

listTxNames = ['ChemoTx', 'HematologicTx', 'RadiationTx', 'ImmunoTx', 'OtherTx']
ptTxRecievedDict = {}
for pt in listKnown:
    # ptTxRecieved = ptDict[pt].txRecieved
    ptChemoLen= len(ptDict[pt].ChemoTx) if type(ptDict[pt].ChemoTx) != bool else 0
    ptHemoLen= len(ptDict[pt].HemeTx) if type(ptDict[pt].HemeTx) != bool else 0
    ptRadLen= len(ptDict[pt].RadTx) if type(ptDict[pt].RadTx) != bool else 0
    ptImmunoLen= len(ptDict[pt].ImmunoTx) if type(ptDict[pt].ImmunoTx) != bool else 0
    ptOtherLen= len(ptDict[pt].OtherTx) if type(ptDict[pt].OtherTx) != bool else 0

    #make a list from these values
    ptTxRecievedList = [ptChemoLen, ptHemoLen, ptRadLen, ptImmunoLen, ptOtherLen]

    #add to dictionary
    ptTxRecievedDict[pt] = ptTxRecievedList

#convert to dataframe
ptTxRecieved_df = pd.DataFrame.from_dict(ptTxRecievedDict, orient='index', columns=listTxNames)

# fits of data
look in old code to see if there are better ways of writing it.

In [263]:
#definitions of functions 

def linear_model_force(x, a):
    return a * x 


In [264]:
def prepDataForFit(dataframe, time, dataColumn, transform = ["none"]):
    dataValues =dataframe[[time, dataColumn]] #collect the data

    #clean data for null or 0 data and remove any negative y values
    #negative y values are not physiologic
    if transform == ["none"]:
        dataValues = dataValues.dropna(axis = 'rows').loc[dataValues[dataColumn]>=0]
        x_values = dataValues[[time]] #time information
        y_values = dataValues[dataColumn] #collect the y values
        baseline = y_values.iloc[0]
        y_values_shift = y_values - baseline
    elif transform == ["ln"]:
        # ln tranform can't handle 0 values
        dataValues = dataValues.dropna(axis = 'rows').loc[dataValues[dataColumn]>=0]
        dataValues[dataColumn] = [value + 1 for value in dataValues[dataColumn]]
        x_values = dataValues[[time]] #time information
        y_values = dataValues[dataColumn] #collect the y values
        y_values_transformed = np.log(y_values)
        baseline = y_values_transformed.iloc[0]
        y_values_shift = y_values_transformed - baseline

    datalength = len(y_values_shift)

    return x_values, y_values_shift, baseline, datalength


In [265]:
#linear model fits using scipy curve_fit where the intercept is the first value
def linear_model_fit(x, y, baseline, datalength):
    if datalength >1:
        #fit the data to a linear model using scipy curve_fit
        popt, pcov = curve_fit(linear_model_force, x, y)
        #get the slope and intercept from the popt array
        coef = popt[0]
        intercept = baseline

        #test predictions
        y_prediction = linear_model_force(x, coef)

        #calculate the mean squared error
        mse = mean_squared_error(y, y_prediction)
        #calculate the coeficient of determination of the fit
        r2 = r2_score(y, y_prediction)
    if datalength  ==1: 
        #if the data length is 1, then set slope to 0, set the baseline to the intercept
        coef = 0
        intercept = baseline
        #now fit the prediction
        y_prediction = linear_model_force(x, coef)
        mse = baseline
        r2 = one_point_r2

    return coef, intercept, mse, r2, y_prediction


In [266]:
#performing the linear fit of lab data 
ptLabsLinFitsDict = {}
for pt in listKnown:
    i = 0
    for lab in filteredLabs:
        ptLabsData = ptDict[pt].labsData[['RelTime(Days)', lab]]
        x_values, y_values_shift, baseline, datalength = prepDataForFit(ptLabsData, 'RelTime(Days)', lab, transform = ["none"])
        x_values_flat = x_values.squeeze()
        y_values_shift_flat = y_values_shift.squeeze()
        #fit to the model
        coef, intercept, mse, r2, y_prediction = linear_model_fit(x_values_flat, y_values_shift_flat, baseline, datalength)
        #save the coef, intercept, mse, r2 to a new dataframe ptLabsFits
        if i ==0: 
            ptLabsFits = pd.DataFrame({ lab + '_linear_coef': [coef], 
                                        lab+'_linear_intercept': [intercept], 
                                        lab+ '_linear_mse': [mse], 
                                        lab+ '_linear_r2': [r2]})
        else:
            ptLabsFits = pd.concat([ptLabsFits, pd.DataFrame({ lab + '_linear_coef': [coef],
                                        lab+'_linear_intercept': [intercept], 
                                        lab+ '_linear_mse': [mse], 
                                        lab+ '_linear_r2': [r2]})], axis = 1)
        i +=1
    ptLabsLinFitsDict[pt] = ptLabsFits



In [267]:
#performing the exponential fits of the lab data
#first transplate data by 1 to correct for 0, then transform using LN, then fit to a linear model
ptLabsExpFitsDict = {}
for pt in listKnown:
    i = 0
    for lab in filteredLabs:
        ptLabsData = ptDict[pt].labsData[['RelTime(Days)', lab]]
        x_values, y_values_shift, baseline, datalength = prepDataForFit(ptLabsData, 'RelTime(Days)', lab, transform = ["ln"])
        x_values_flat = x_values.squeeze()
        y_values_shift_flat = y_values_shift.squeeze()
        #fit to the model
        coef, intercept, mse, r2, y_prediction = linear_model_fit(x_values_flat, y_values_shift_flat, baseline, datalength)
        #save the coef, intercept, mse, r2 to a new dataframe ptLabsFits
        if i ==0: 
            ptLabsFits = pd.DataFrame({ lab + '_exp_coef': [coef], 
                                        lab+'_exp_intercept': [intercept], 
                                        lab+ '_exp_mse': [mse], 
                                        lab+ '_exp_r2': [r2]})
        else:
            ptLabsFits = pd.concat([ptLabsFits, pd.DataFrame({ lab + '_exp_coef': [coef],
                                        lab+'_exp_intercept': [intercept], 
                                        lab+ '_exp_mse': [mse], 
                                        lab+ '_exp_r2': [r2]})], axis = 1)
        i +=1
    ptLabsExpFitsDict[pt] = ptLabsFits



## fits for the note data

In [268]:
#performing the linear fit of lab data 
ptNotesLinFitsDict = {}
noteTimeColumn = 'time from diagnosis to note (months)'

for pt in listKnown:
    i = 0
    for lab in noteColumns:
        ptNotesData = ptDictMDnotesScored[pt][[noteTimeColumn, lab]]
        x_values, y_values_shift, baseline, datalength = prepDataForFit(ptNotesData, noteTimeColumn, lab, transform = ["none"])
        x_values_flat = x_values.squeeze()
        y_values_shift_flat = y_values_shift.squeeze()
        #fit to the model
        coef, intercept, mse, r2, y_prediction = linear_model_fit(x_values_flat, y_values_shift_flat, baseline, datalength)
        #save the coef, intercept, mse, r2 to a new dataframe ptNotesFits
        if i ==0: 
            ptNotesFits = pd.DataFrame({ lab + '_linear_coef': [coef], 
                                        lab+'_linear_intercept': [intercept], 
                                        lab+ '_linear_mse': [mse], 
                                        lab+ '_linear_r2': [r2]})
        else:
            ptNotesFits = pd.concat([ptNotesFits, pd.DataFrame({ lab + '_linear_coef': [coef],
                                        lab+'_linear_intercept': [intercept], 
                                        lab+ '_linear_mse': [mse], 
                                        lab+ '_linear_r2': [r2]})], axis = 1)
        i +=1
    ptNotesLinFitsDict[pt] = ptNotesFits



In [269]:
#performing the linear fit of lab data 
ptNotesExpFitsDict = {}
noteTimeColumn = 'time from diagnosis to note (months)'

for pt in listKnown:
    i = 0
    for lab in noteColumns:
        ptNotesData = ptDictMDnotesScored[pt][[noteTimeColumn, lab]]
        x_values, y_values_shift, baseline, datalength = prepDataForFit(ptNotesData, noteTimeColumn, lab, transform = ["ln"])
        x_values_flat = x_values.squeeze()
        y_values_shift_flat = y_values_shift.squeeze()
        #fit to the model
        coef, intercept, mse, r2, y_prediction = linear_model_fit(x_values_flat, y_values_shift_flat, baseline, datalength)
        #save the coef, intercept, mse, r2 to a new dataframe ptNotesFits
        if i ==0: 
            ptNotesFits = pd.DataFrame({ lab + '_exp_coef': [coef], 
                                        lab+'_exp_intercept': [intercept], 
                                        lab+ '_exp_mse': [mse], 
                                        lab+ '_exp_r2': [r2]})
        else:
            ptNotesFits = pd.concat([ptNotesFits, pd.DataFrame({ lab + '_exp_coef': [coef],
                                        lab+'_exp_intercept': [intercept], 
                                        lab+ '_exp_mse': [mse], 
                                        lab+ '_exp_r2': [r2]})], axis = 1)
        i +=1
    ptNotesExpFitsDict[pt] = ptNotesFits



# compile the dataframes for use in SVM 

# data visualizations

## completeness of the dataframes

## event plots of the data
include the echo date

## plot of selected patient labs and fit of data